In [ ]:
!pip install easyocr --no-deps # Colab already has all dependencies
!pip install python-bidi
!pip install   tensorflow
!pip install  tensorflow-text
!pip install  tf-models-official

     |████████████████████████████████| 63.2MB 42kB/s 
     |████████████████████████████████| 4.3MB 4.1MB/s 
     |████████████████████████████████| 1.6MB 3.9MB/s 
     |████████████████████████████████| 358kB 17.7MB/s 
     |████████████████████████████████| 215kB 18.3MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 686kB 18.8MB/s 
     |████████████████████████████████| 1.2MB 25.7MB/s 
     |████████████████████████████████| 645kB 27.0MB/s 
     |████████████████████████████████| 38.2MB 81kB/s 
     |████████████████████████████████| 102kB 9.4MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22258 sha256=bc7f33007c9f74de8ae0fb0baa564c6dd5482203471a2750c8398c585e29c6f8
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl s

In [54]:
import itertools
import os, glob

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

from datetime import datetime, timedelta

import zipfile, shutil
import re

In [53]:

import os
import re

import easyocr
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

text
from official import nlp
from official.nlp import bert
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks


In [ ]:
import yaml
yaml_file = "/content/drive/MyDrive/config.yaml"
dataset_name = "test_data"
# parse configuration
with open(yaml_file, 'r', encoding='utf-8') as yaml_file:
  task_config = yaml.load(yaml_file.read(), Loader=yaml.SafeLoader)
data_path="/content/drive/MyDrive/store-type-recognition/split/"

train_dataset, val_dataset, test_dataset = data_path+"train",data_path+"val",data_path+"test",

# ocr_bert = OCRBertClassifier(task_config, train_dataset, val_dataset, test_dataset)



class BertDataset:
    def __init__(self, x, y, label_map,data_size=1):
        self.x = self.preprocess(tf.constant(x))
        self.y = tf.constant(y)
        self.label_map = label_map
        self.data_size= data_size
    def preprocess(self,x):
      # wget https://tfhub.dev/tensorflow/bert_zh_preprocess/3
      text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
      preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_zh_preprocess/3")
      encoder_inputs = preprocessor(text_input)
      preprocess_model = tf.keras.Model(text_input, encoder_inputs)
      return preprocess_model(x)



In [ ]:
import csv
from tqdm import tqdm
def remove_special_chars(input):
    input = re.sub('[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~\s]+', "", input)
    return re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])", "", input)
ocr_reader=easyocr.Reader(['ch_sim', 'en'],gpu=True)


def generate_csv(data_type):
  x_array = []
  y_array = []
  label_map = {}
  dataset_path=os.path.join(data_path,data_type)
  with open(f'/content/drive/MyDrive/store_type_recognition_{data_type}.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvwriter.writerow(["image","label","label_id","text"])
    for label_index, label in enumerate(os.listdir(dataset_path)):
      label_dir = os.path.join(dataset_path, label)
      for img in tqdm(os.listdir(label_dir)):
        img = os.path.join(label_dir, img)
        # TODO batch input
        bounds = ocr_reader.readtext(img,batch_size=2)
        if not bounds:
          print("no bounds")
        orig_str = ''.join([ret[1] for ret in bounds])
        curr_str = remove_special_chars(orig_str)
        csvwriter.writerow([img,label,label_map[label],curr_str])
# generate_csv("val")
generate_csv("train")
# generate_csv("test")

KeyboardInterrupt: ignored

In [ ]:
import re
import os
label_map = {'domestic-services': 1, 'food': 3, 'others': 0, 'store': 2}

def colab_data(data_type):
  data_path = "/content/drive/MyDrive/store-type-recognition/split"
  dataset_path = os.path.join(data_path, data_type)
  df=pd.read_csv(f"/content/drive/MyDrive/store_type_recognition_{data_type}.csv")
  df=df.dropna()
  return df

data=colab_data("train")
print(data)
char=data.iloc[158]["text"]
print(char)
def is_rare_name(string):
  try:
    string.encode("gb2312")
  except UnicodeEncodeError:
    return True
  return False
for char in data["text"]:

  if is_rare_name(char):
    print(is_rare_name(char))


                                                  image  ...                                               text
0     /data/AutoML_compete/Store-type-recognition/sp...  ...                             本濠房产赛簟兵热线电话廖先生菰濠房屋租售售租
1     /data/AutoML_compete/Store-type-recognition/sp...  ...                                     银行很愆纤上理财轻妾全上上幺
2     /data/AutoML_compete/Store-type-recognition/sp...  ...                                              链家万超元
3     /data/AutoML_compete/Store-type-recognition/sp...  ...                                      盛美置皿广一父盥一怄厂刀工
4     /data/AutoML_compete/Store-type-recognition/sp...  ...                                       佳乐房产佳乐房产为您服售
...                                                 ...  ...                                                ...
1620  /data/AutoML_compete/Store-type-recognition/sp...  ...  杰灭汊弧乒竹蹂凤记鳘倌空诹麦位足购开锬禾风记经络养生馆开肯沭足蔬通经络生姜穴位贴姜艾艾贴蓄隳...
1621  /data/AutoML_compete/Store-type-recognition/sp...  ...              唯爱美甲美睫半永久纹绣壬兄唯爱一起值酋瀵平适匮弛凇睫牮滩丞丢

In [59]:
import pandas as pd
import os
import jieba


label_map = {'domestic-services': 1, 'food': 3, 'others': 0, 'store': 2}

def colab_data(data_type):
  data_path = "/content/drive/MyDrive/store-type-recognition/split"
  dataset_path = os.path.join(data_path, data_type)
  df=pd.read_csv(f"/content/drive/MyDrive/store_type_recognition_{data_type}.csv")
  df=df.sample(frac=1)
  # 用""填空
  df=df.fillna(value="")
  #分词
  origin_text=df['text']
  for line in df:
    for vocb in jieba.cut(char, cut_all=True):
      vocv_row={"image":}
      config_pd = config_pd.append(best_config, ignore_index=True)
      df.append
    x=df['text'].apply(lambda char : "".join(list(jieba.cut(char, cut_all=True))))
  # 丢弃空值
  # df=df.dropna()
  # x=df["text"]
  # print(x,y)
  
  
  # # 找到有空值的图像
  y=df['label'].apply(lambda x : label_map[x])

  x_null=df[df.isnull().T.any()]
  assert x_null.isnull
  print(x_null)
  print(x)
  # bert_dataset = BertDataset(x=x.tolist(), y=y.tolist(),label_map=label_map,data_size=len(x))
  # return bert_dataset

# val_dataset,train_dataset = colab_data("val"),colab_data("train")
test_dataset=colab_data("test")


Empty DataFrame
Columns: [image, label, label_id, text]
Index: []
160                                        溟嗲脚沟炎巫指甲眼兰宅颗帧
115                                         中围联通伏龙华华阳菖业厅
177                                                 寨源干屎
331        干里香匦笫冢门店戍都睁和路店加翌热线饨面黄焖鸡米饭附招工吁囊吕薹冼香氓壬吃愠饨去千里千里香
417                                乐山钵钵鸡小郎酒自冷吃免小郎酒此盛宦抹小尤
                             ...                        
251    雅管綮哑联系联系电话电话蒙蒡蜚位出租睢茗茕业高新新茶新荼上市竹叶竹叶青畿碧螺碧螺春竹卟眨丝阜...
427    糖宝玉香饭店店主主营冒莱干锅炒慕娆特色干锅黧躐炒菜冒锅锅锅干锅拼盘炒河粉冒牌冒菜特推冒烤鸭大...
128                                    舒肯苴美肌站舒设权舸鹉间书芒泪汗水
9                                                中信建投证券儿
277                                                    众
Name: text, Length: 449, dtype: object


In [57]:
#分词
test_dataset.x

{'input_mask': <tf.Tensor: shape=(449, 128), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(449, 128), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(449, 128), dtype=int32, numpy=
 array([[ 101, 3362, 4197, ...,    0,    0,    0],
        [ 101, 4462,  102, ...,    0,    0,    0],
        [ 101, 4510, 5753, ...,    0,    0,    0],
        ...,
        [ 101, 1488, 1995, ...,    0,    0,    0],
        [ 101,  100,  100, ...,    0,    0,    0],
        [ 101, 6756, 1075, ...,    0,    0,    0]], dtype=int32)>}

In [ ]:

bert_config_dict = {
    "attention_probs_dropout_prob": 0.1,
    "directionality": "bidi",
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "max_position_embeddings": 512,
    "num_attention_heads": 12,
    "num_hidden_layers": 12,
    "pooler_fc_size": 768,
    "pooler_num_attention_heads": 12,
    "pooler_num_fc_layers": 3,
    "pooler_size_per_head": 128,
    "pooler_type": "first_token_transform", 
    "type_vocab_size": 2,
    "vocab_size": 21128
}
print(train_dataset.y[:10],val_dataset.y[:10],test_dataset.y[:10])
print((train_dataset.data_size), val_dataset, len(train_dataset.label_map))
bert_config = bert.configs.BertConfig.from_dict(bert_config_dict)
bert_classifier, bert_encoder = bert.bert_models.classifier_model(bert_config,num_labels=len(train_dataset.label_map))
epochs=5
batch_size=32
lr=2e-5                                                            
checkpoint = tf.train.Checkpoint(encoder=bert_encoder)
# pip install gsutil
# gsutil cp -R gs://cloud-tpu-checkpoints/bert/v3/chinese_L-12_H-768_A-12.tar.gz chinese_L-12_H-768_A-12

# restore the checkpoint
checkpoint.read(os.path.join('/content/drive/MyDrive/chinese_L-12_H-768_A-12/tmp/temp_dir/raw/', 'bert_model.ckpt')).assert_consumed()
# gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/chinese_L-12_H-768_A-12.tar.gz"
# tf.io.gfile.copy(gs_folder_bert, "./chinese_L-12_H-768_A-12.tar.gz")

# !mkdir -p chinese_L-12_H-768_A-12 && tar -xf ./chinese_L-12_H-768_A-12.tar.gz -C ./chinese_L-12_H-768_A-12

# restore the checkpoint
# checkpoint.read(os.path.join('./chinese_L-12_H-768_A-12/tmp/temp_dir/raw/', 'bert_model.ckpt')).assert_consumed()
train_data_size = (train_dataset.data_size)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

# creates an optimizer with learning rate schedule
optimizer = nlp.optimization.create_optimizer(lr, num_train_steps=num_train_steps,
                                              num_warmup_steps=warmup_steps)

metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

bert_classifier.compile(optimizer=optimizer, loss=loss, metrics=metrics)

bert_classifier.fit(
    train_dataset.x, train_dataset.y,
    validation_data=(val_dataset.x, val_dataset.y),
    batch_size=batch_size,
    epochs=epochs)
results = bert_classifier.evaluate(test_dataset.x, test_dataset.y)
print(results)
tf.saved_model.save(bert_classifier, export_dir="./saved_model-use-emtpystr-fill")

 

tf.Tensor([1 2 1 2 3 3 3 3 3 1], shape=(10,), dtype=int32) tf.Tensor([1 3 3 3 3 3 3 3 3 1], shape=(10,), dtype=int32) tf.Tensor([3 1 3 3 3 3 2 1 1 2], shape=(10,), dtype=int32)
1625 <__main__.BertDataset object at 0x7f1482808750> 4
Epoch 1/5


51/51 [==============================] - 54s 803ms/step - loss: 1.1917 - accuracy: 0.4449 - val_loss: 0.8905 - val_accuracy: 0.6481
Epoch 2/5
51/51 [==============================] - 40s 793ms/step - loss: 0.8183 - accuracy: 0.6726 - val_loss: 0.6978 - val_accuracy: 0.7531
Epoch 3/5
51/51 [==============================] - 40s 785ms/step - loss: 0.5872 - accuracy: 0.7865 - val_loss: 0.6661 - val_accuracy: 0.7222
Epoch 4/5
51/51 [==============================] - 40s 789ms/step - loss: 0.4591 - accuracy: 0.8480 - val_loss: 0.6833 - val_accuracy: 0.7346
Epoch 5/5
15/15 [==============================] - 4s 247ms/step - loss: 0.3995 - accuracy: 0.8686
[0.39946186542510986, 0.8685969114303589]



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)

FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: ./saved_model-use-emtpystr-fill/assets


INFO:tensorflow:Assets written to: ./saved_model-use-emtpystr-fill/assets


# 单纯跳过空行
空行数 val 1 train 22  test7
```
51/51 [==============================] - 54s 787ms/step - loss: 1.2442 - accuracy: 0.4017 - val_loss: 0.9536 - val_accuracy: 0.6584
Epoch 2/5
51/51 [==============================] - 40s 784ms/step - loss: 0.8754 - accuracy: 0.6450 - val_loss: 0.6879 - val_accuracy: 0.7267
Epoch 3/5
51/51 [==============================] - 39s 773ms/step - loss: 0.6197 - accuracy: 0.7611 - val_loss: 0.6713 - val_accuracy: 0.7391
Epoch 4/5
51/51 [==============================] - 40s 779ms/step - loss: 0.4725 - accuracy: 0.8172 - val_loss: 0.6613 - val_accuracy: 0.7391
Epoch 5/5
51/51 [==============================] - 40s 777ms/step - loss: 0.3838 - accuracy: 0.8777 - val_loss: 0.6660 - val_accuracy: 0.7516
14/14 [==============================] - 4s 261ms/step - loss: 0.4056 - accuracy: 0.8643
[0.4056240916252136, 0.8642534017562866]
```

# 使用空字符""替换
```
51/51 [==============================] - 54s 803ms/step - loss: 1.1917 - accuracy: 0.4449 - val_loss: 0.8905 - val_accuracy: 0.6481
Epoch 2/5
51/51 [==============================] - 40s 793ms/step - loss: 0.8183 - accuracy: 0.6726 - val_loss: 0.6978 - val_accuracy: 0.7531
Epoch 3/5
51/51 [==============================] - 40s 785ms/step - loss: 0.5872 - accuracy: 0.7865 - val_loss: 0.6661 - val_accuracy: 0.7222
Epoch 4/5
51/51 [==============================] - 40s 789ms/step - loss: 0.4591 - accuracy: 0.8480 - val_loss: 0.6833 - val_accuracy: 0.7346
Epoch 5/5
51/51 [==============================] - 40s 787ms/step - loss: 0.3869 - accuracy: 0.8818 - val_loss: 0.6666 - val_accuracy: 0.7531
15/15 [==============================] - 4s 247ms/step - loss: 0.3995 - accuracy: 0.8686
[0.39946186542510986, 0.8685969114303589]
```


In [ ]:
# !rm -rf /content/drive/MyDrive/store-type-recognition
# !cd /content/drive/MyDrive && unzip /content/drive/MyDrive/store-type-recognition.zip
!ls /content/drive/MyDrive/store-type-recognition/split/train/food/ |wc -l
!ls /content/drive/MyDrive/store-type-recognition/split/test/store/ |wc -l
!ls /content/drive/MyDrive/store-type-recognition/split/val/food/ |wc -l
!ls /content/store-type-recognition/split/test/store |wc -l
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/chinese_L-12_H-768_A-12.tar.gz"
tf.io.gfile.copy(gs_folder_bert, "/content/drive/MyDrive/chinese_L-12_H-768_A-12.tar.gz")
!cd /content/drive/MyDrive &&tar -zxvf chinese_L-12_H-768_A-12.tar.gz -C chinese_L-12_H-768_A-12
!nvidia-smi


569
123
62
ls: cannot access '/content/store-type-recognition/split/test/store': No such file or directory
0
tmp/temp_dir/raw/
tmp/temp_dir/raw/vocab.txt
tmp/temp_dir/raw/bert_model.ckpt.index
tmp/temp_dir/raw/bert_model.ckpt.data-00000-of-00001
tmp/temp_dir/raw/bert_config.json
Thu Jun 24 12:32:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    72W / 149W |   1148MiB / 11

In [ ]:
!kill -9 60 

In [ ]:
import os , shutil
data_path = "/content/drive/MyDrive/store-type-recognition/split/test"
labels = os.listdir(data_path)
data_path="/content/store-type-recognition/split/test/store"
for label in os.listdir(data_path):
  label_image = os.path.join(data_path, label)
  shutil.copy(os.path.join(label_image),os.path.join("/content/drive/MyDrive/store-type-recognition/split/test/store",label))

In [ ]:
for label in labels:
  if label =="food":
    print(label)
    label_dir = os.path.join(data_path, label)
    val_label_dir=label_dir.replace("test","val")
    if not os.path.exists(val_label_dir):
        os.mkdir(val_label_dir)
    with open(f"/content/drive/MyDrive/{label}.txt", "r")as f:
        images_name = f.readlines()
        for img in images_name:
          number = re.findall(r"\d+", img)[0].strip()
          for train_img in os.listdir(label_dir):
            train_number=re.findall(r"\d+", train_img)[0].strip()
            if number == train_number:
              print(number,train_number)
              print(os.path.join(label_dir,train_img),os.path.join(val_label_dir,train_img))
              # os.remove(os.path.join(label_dir,train_img))
              shutil.move(os.path.join(label_dir,train_img),os.path.join(val_label_dir,train_img))
              print(os.path.join(label_dir,train_img))